```bash
{
conda activate
cd ~/link/res_publish/run
jupyter nbconvert info_model_gene.ipynb --to python

conda activate publish
nohup python info_model_gene.py > nohup_info_model_gene &
sleep 30 && rm info_model_gene.py
conda activate
clear
}
jobs
```

In [1]:
import sys
from pathlib import Path
p_root = Path('~/link/res_publish').expanduser()
None if str(p_root) in sys.path else sys.path.append(str(p_root))

In [2]:
from func import *

import utils as ut
from utils.general import *
pl = ut.pl

p_res_model_gene  = p_res.joinpath('model_gene')
p_res_model_gene.mkdir(exist_ok=True,parents=True)

2024-08-19 02:05:17.713959: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.



-------------------------func_help-------------------------
> parameter
    p_root	[name] res_publish
        p_run, p_plot, p_res, p_cache, p_pdf
    p_df_varmap
    map_sp_reverse
    rng
> run
    run_cross_species_models
    h5ad_to_mtx
    load_adata
    get_path_varmap
    find_path_from_para
    load_normalized_adata

> res
    get_test_result_df
    get_res_obs
    get_adata_umap
    show_umap

> plot
    get_color_map
    show_color_map
    show_color
    plot_umap
    savefig



In [3]:
Tensor = csMAHN.pp.Tensor
torch = csMAHN.pp.torch
with Block("run_csMAHN_info_model_gene"):
    def run_csMAHN_info_model_gene(
        path_adata1,
        path_adata2,
        key_class1,
        key_class2,
        sp1,
        sp2,
        tissue_name,
        path_varmap,
        aligned=False,
        resdir_tag=".",
        resdir=Path('.'),
        limite_func=lambda adata1, adata2: (adata1, adata2),
        **kvargs
    ):
        """
        version = 0.0.9
        kvargs:
            n_epochs:
                default,[100, 200, 300]
                stages,即res_0,res_1，res_2 的 epochs
                累加制，res_0,res_1，res_2,实际epochs分别为100,300,600
                故最终epochs为stages之和
                stages = kvargs.setdefault("n_epochs",[100, 200, 300])
    
            is_1v1: bool
                default,False
            n_hvgs:
                default,2000
            n_degs:
                default,50
        """
        homo_method = 'biomart'
        n_hvgs = kvargs.setdefault('n_hvgs', 2000)
        n_degs = kvargs.setdefault('n_degs', 50)
        seed = 123
        stages = kvargs.setdefault(
            'n_epochs', [
                100, 200, 200])  # [200, 200, 200]
        nfeats = kvargs.setdefault('nfeats', 64)  # 64  # embedding size #128
        hidden = kvargs.setdefault('hidden', 64)  # 64  # 128
        input_drop = 0.2
        att_drop = 0.2
        residual = True
    
        threshold = 0.9  # 0.8
        lr = 0.01  # lr = 0.01
        weight_decay = 0.001
        patience = 100
        enhance_gama = 10
        simi_gama = 0.1
    
        dsnames = (
            '{}_{}'.format(
                tissue_name, sp1), '{}_{}'.format(
                tissue_name, sp2))
        assert key_class1 == key_class2, "key_class is not equal"
        key_class = key_class1
    
        # make file to save
        resdir_tag = "{}_{}-corss-{};{}".format(tissue_name, sp1, sp2, resdir_tag) if len(
            resdir_tag) > 0 else "{}_{}-corss-{}".format(tissue_name, sp1, sp2)
        # curdir = os.path.join()
        resdir = Path(resdir).joinpath(resdir_tag)
        model_dir = resdir.joinpath('model_')
        figdir = resdir.joinpath('figs')
        
        resdir.mkdir(exist_ok=True, parents=True)
        # [_.mkdir(exist_ok=True, parents=True)
        #  for _ in [resdir, model_dir, figdir]]
        # [resdir.joinpath('res_{}'.format(i)).mkdir(
        #     exist_ok=True, parents=True) for i in range(len(stages))]
    
        checkpt_file = model_dir.joinpath("mutistages")
    
        # is finish
        # p_finish = Path(resdir).joinpath("finish")
        # if p_finish.exists():
        if resdir.joinpath('info_mode_gene.json').exists() and 
        np.setdiff1d(
            "reference_hvgs,reference_degs,reference_higs,query_hvgs,query_degs,query_higs,reference_gene_nodes,query_gene_nodes,ortholog_one2one,ortholog_one2many,ortholog_many2many".split(','),
            list(json.loads(resdir.joinpath('info_mode_gene.json').read_text()).keys())
        ).size == 0 :
            print(
                "[has finish]{} {}".format(
                    time.strftime('%y%m%d-%H%M', time.localtime()), resdir.name
                ))
            return
        else:
            pass
        print(
            "[start]{} {}".format(
                time.strftime('%y%m%d-%H%M', time.localtime()),
                resdir.name
            ))
    
        finish_content = ["[strat] {}".format(time.time())]
        print('[path_varmap] {}'.format(path_varmap))
        adata_raw1 = load_adata(path_adata1)
        adata_raw2 = load_adata(path_adata2)
        if key_class not in adata_raw2.obs.columns:
            adata_raw2.obs[key_class] = ''
        adata_raw1.obs[key_class] = adata_raw1.obs[key_class].astype(str)
        adata_raw2.obs[key_class] = adata_raw2.obs[key_class].astype(str)
        # limite 进一步对adata进行限制，默认不操作直接返回
        adata_raw1, adata_raw2 = limite_func(
            adata_raw1, adata_raw2
        )
        # group_counts_unalign.csv
        pd.concat([adata_raw1.obs[key_class].value_counts(),
                   adata_raw2.obs[key_class].value_counts(),],
                  axis=1, keys=dsnames,)
        # .to_csv(
        #     resdir.joinpath("group_counts_unalign.csv"), index=True
        # )
        # 仅保留公共细胞类群
        if aligned:
            adata_raw1, adata_raw2 = csMAHN.pp.aligned_type(
                [adata_raw1, adata_raw2], key_class
            )
    
        # group_counts.csv
        temp = pd.concat([adata_raw1.obs[key_class].value_counts(),
                          adata_raw2.obs[key_class].value_counts(),],
                         axis=1, keys=dsnames)
        print(temp)
        # temp.to_csv(resdir.joinpath("group_counts.csv"), index=True)
        # adata_raw1.obs.to_csv(resdir.joinpath("obs_ref.csv"), index=True)
        # adata_raw2.obs.to_csv(resdir.joinpath("obs_que.csv"), index=True)
    
        # homo = pd.read_csv(path_varmap)
        homo = pd.read_csv(path_varmap, usecols=range(3))
        homo.columns = ["gn_ref", "gn_que", "homology_type"]
        if kvargs.setdefault("is_1v1", False):
            homo = get_1v1_matches(homo)
            homology_parameter = get_homology_parameters(
                adata_raw1, adata_raw2, homo)
            print("""
    [homology one2one]find {homology_one2one_find} genes
    [homology one2one]use {homology_one2one_use} genes""".format(
                **homology_parameter))
            kvargs.update(homology_parameter)
    
        kvargs.update({'path_adata1': str(path_adata1),
                       'path_adata2': str(path_adata2),
                       'key_class1': key_class1,
                       'key_class2': key_class2,
                       'sp1': sp1,
                       'sp2': sp2,
                       'tissue_name': tissue_name,
                       'path_varmap': str(path_varmap),
                       'aligned': aligned,
                       'resdir_tag': resdir_tag,
                       'resdir': str(resdir),
                      'n_hvgs': n_hvgs,
                       'n_degs': n_degs,
                       'nfeats': nfeats,
                       'hidden': hidden
                       })
        resdir.joinpath("kvargs.json").write_text(dumps(kvargs))
        print(
            """Task: refernece:{} {} cells x {} gene -> query:{} {} cells x {} gene in {}""".format(
                dsnames[0],
                adata_raw1.shape[0],
                adata_raw1.shape[1],
                dsnames[1],
                adata_raw2.shape[0],
                adata_raw2.shape[1],
                tissue_name))
    
        start = time.time()
        finish_content.append("[finish before run] {}".format(time.time()))
        # knn时间较长
        print("\n[process_for_graph]\n".center(100, '-'))
        adatas, features_genes, nodes_genes, scnets, one2one, n2n = csMAHN.pp.process_for_graph(
            [adata_raw1, adata_raw2], homo, key_class, 'leiden', n_hvgs=n_hvgs, n_degs=n_degs,
            resdir=resdir)
        g, inter_net, one2one_gene_nodes_net, cell_label, n_classes, list_idx = csMAHN.pp.make_graph(
            adatas, aligned, key_class, features_genes, nodes_genes, scnets,
            one2one, n2n, has_mnn=True, seed=seed,resdir=resdir)
        end = time.time()
        # 包括预处理时间
        print('Times preprocess for graph:{:.2f}'.format(end - start))
        print('\n[end][run_csMAHN_info_model_gene]\n'.center(100,'-'))

with Block("[redefine] csMAHN.pp.get_adj_gene"):
    def _func(nodes_genes, homo,**kvarg):
        """
        Outputs a gene aparse adjacent matrix
        :param homo: genes homology relation (select one2one or many2many to control)
        """
        reference_gene_nodes = nodes_genes[0]
        query_gene_nodes = nodes_genes[1]
        adj = csMAHN.pp.match_bigraph(reference_gene_nodes, query_gene_nodes, homo)
        print("--------------homo edges---------------")
        print(adj.iloc[:, 2].value_counts())
        
        # [redefine][add]
        with Block("info_mode_gene.json",context=dict(
            p_out = kvarg['resdir'].joinpath('info_mode_gene.json'),
            data = {}
        )) as context:
            if context.p_out.exists():
                context.data.update(json.loads(context.p_out.read_text()))
            context.data.update(
                adj.iloc[:, 2].value_counts().to_frame()\
                    .reset_index(names='value')\
                    .pipe(ut.df.to_dict,'value','count'))
            context.p_out.write_text(json.dumps(context.data))
    
        gn = np.append(reference_gene_nodes, query_gene_nodes)
        gnind_1 = pd.DataFrame(data=np.arange(reference_gene_nodes.size)[None, :], columns=reference_gene_nodes)
        gnind_2 = pd.DataFrame(data=np.arange(gn.size)[None, :], columns=gn)
        gnind_2 = gnind_2.iloc[:, len(reference_gene_nodes):]
        a = np.array(adj.iloc[:, 0]).tolist()
        b = np.array(adj.iloc[:, 1]).tolist()
        reference_indic = pd.Index(gnind_1[a].values.flatten())
        query_indic = pd.Index(gnind_2[b].values.flatten())
        indice1 = np.vstack((reference_indic, query_indic))
        indice2 = np.vstack((query_indic, reference_indic))
        gene_adj = np.concatenate((indice1, indice2), axis=1)
    
        return torch.LongTensor(gene_adj)

    csMAHN.pp.get_adj_gene = _func
    del _func

with Block("[redefine] csMAHN.pp.select_gene_nodes"):
    def _func(reference_all_gene, query_all_gene, higs_reference, higs_query, homo,**kvarg):
        # 物种1的higs与物种2的higs在物种1的同源 联合
        query_homo = homo[homo.iloc[:, 1].isin(higs_query)].iloc[:, 0]
        reference_gene_nodes = np.union1d(query_homo, higs_reference)
    
        reference_homo = homo[homo.iloc[:, 0].isin(higs_reference)].iloc[:, 1]
        query_gene_nodes = np.union1d(reference_homo, higs_query)
    
        # biomart和数据集基因名的一致性筛选
        reference_gene_nodes = np.intersect1d(reference_all_gene, reference_gene_nodes)
        query_gene_nodes = np.intersect1d(query_all_gene, query_gene_nodes)
        print("--------------gene nodes info---------------")
        print("num of reference_gene_node is {0}".format(len(reference_gene_nodes)))
        print("num of query_gene_node is {0}".format(len(query_gene_nodes)))
        
        # [redefine][add]
        with Block("info_mode_gene.json",context=dict(
            p_out = kvarg['resdir'].joinpath('info_mode_gene.json'),
            data = {}
        )) as context:
            if context.p_out.exists():
                context.data.update(json.loads(context.p_out.read_text()))
            context.data.update(
                reference_gene_nodes=len(reference_gene_nodes),
                query_gene_nodes=len(query_gene_nodes)
                )
            context.p_out.write_text(json.dumps(context.data))

        return reference_gene_nodes, query_gene_nodes
    
    csMAHN.pp.select_gene_nodes = _func
    del _func
    
with Block("[redefine] csMAHN.pp.process_for_graph"):
    def _func(adatas,
                  homo,
                  key_class,
                  key_clust,
                  relation='many',
                  add_no1v1=True,
                  n_pcs=30,
                  n_hvgs=2000,
                  n_degs=50,
                  use_scnets=True,
                  n_neighbors_scnet=5,
                  n_neighbors_clust=20,
                  reso=0.4,
                  degs_method="wilcoxon",
                  **kvarg
                  ):
        
        print('*'*75,'\n[redefind] csMAHN.pp.process_for_graph\n{}\n'.format(kvarg),'*'*75)
        # process homo relation
        one2one, n2n = csMAHN.pp.biomart_process(homo)
        
        # process for adata
        params_preproc = dict(
            copy=True,
            target_sum=None,
            n_hvgs=n_hvgs,
            n_pcs=n_pcs,
            n_neighbors=n_neighbors_scnet,
        )
        time1 = time.time()
        adata1 = csMAHN.pp.preprocess_for_adata(adatas[0], **params_preproc)
        adata2 = csMAHN.pp.preprocess_for_adata(adatas[1], **params_preproc)
        time2 = time.time()
        print(f'the time2 of processing adatas is {time2-time1}')
        if use_scnets:
            scnets = [csMAHN.pp.get_intra_net_from_adata(adata1),
                      csMAHN.pp.get_intra_net_from_adata(adata2)]
            scnets[0].data[:] = 1
            scnets[1].data[:] = 1
        else:
            scnets = None
        
        reference_hvgs, query_hvgs = csMAHN.pp.get_hvgs(adata1), csMAHN.pp.get_hvgs(adata2)
        
        time1 = time.time()
        # 或许需要重新knn
        clust_lbs2 = csMAHN.pp.get_leiden_labels(adata2,
                                       n_neighbors=n_neighbors_clust,
                                       reso=reso,
                                       neighbors_key='clust',
                                       key_added='leiden',
                                       copy=False)
        time2 = time.time()
        print(f'the time of leiden is {time2 - time1}')
        
        
        adatas[1].obs[key_clust] = clust_lbs2
        
        reference_degs = csMAHN.pp.get_degs(adata1, groupby=key_class, n_degs=n_degs, method=degs_method)
        query_degs = csMAHN.pp.get_degs(adata2, groupby=key_clust, n_degs=n_degs, method=degs_method)
        time3 = time.time()
        print(f'the time of degs is {time3 - time2}')
        reference_higs = np.union1d(reference_hvgs, reference_degs)
        query_higs = np.union1d(query_hvgs, query_degs)
        _adatas = [adata1, adata2]
        
        print("--------------hvgs, degs info---------------")
        print("num of reference_hvgs,reference_degs,reference_higs are {0},{1},{2}".format(len(reference_hvgs),
                                                                                           len(reference_degs),
                                                                                           len(reference_higs)))
        print("num of query_hvgs,query_degs,query_higs are {0},{1},{2}".format(len(query_hvgs), len(query_degs),
                                                                                   len(query_higs)))
        # [redefine][add]
        with Block("info_mode_gene.json",context=dict(
            p_out = kvarg['resdir'].joinpath('info_mode_gene.json'),
            data = {}
        )) as context:


            if context.p_out.exists():
                context.data.update(json.loads(context.p_out.read_text()))
            context.data.update(
                reference_hvgs=len(reference_hvgs),
                reference_degs=len(reference_degs),
                reference_higs=len(reference_higs),
                query_hvgs=len(query_hvgs),
                query_degs=len(query_degs),
                query_higs=len(query_higs),
                )
            context.p_out.write_text(json.dumps(context.data))
        
        # nodes gene
        # 使用one2one,n2n选择加入的同源关系类型
        gene_relation = n2n if relation == 'many' else one2one
        reference_all_gene, query_all_gene = adata1.raw.var.index.tolist(), adata2.raw.var.index.tolist()
        reference_gene_nodes, query_gene_nodes = csMAHN.pp.select_gene_nodes(reference_all_gene, query_all_gene,
                                                                   reference_higs, query_higs, gene_relation,
                                                                             **kvarg)
        nodes_genes = [reference_gene_nodes, query_gene_nodes]
        # feature gene
        features_genes = csMAHN.pp.get_feature_genes(reference_all_gene, query_all_gene, reference_degs, query_degs, one2one , add_no1v1=add_no1v1, n2n=n2n)
        
        return _adatas, features_genes, nodes_genes, scnets, one2one, n2n

    csMAHN.pp.process_for_graph = _func
    del _func

with Block("[redefine] csMAHN.pp.make_graph"):
    def _func(adatas,
                   aligned,
                   key_class,
                   features_genes,
                   nodes_genes,
                   scnets,
                   one2one, n2n,
                   gene_embedding=None,
                   has_mnn=False,
                   graph_type='dgl',
                   seed=123,**kvarg):
        """
        Load cell expression matrix ,cell label, homo information to camputa cell hvgs, degs
        adata should be normalized
        """
    
        # normalized and log
        norm_counts = csMAHN.pp.get_counts_from_adatas(adatas)
        # scale
    
        # def _zscore(adata):
        #     """
        #     Adata has been normalized
        #     """
        #     _adata = adata.copy()
        #     sc.pp.scale(_adata)
        #     return pd.DataFrame(_adata.X, columns=_adata.var.index)
        #
        # scale_counts = [_zscore(adata) for adata in adatas]
        reference_labels, query_labels = csMAHN.pp.get_labels_from_adatas(adatas, key_class)
    
        cell_feature = csMAHN.pp.get_feature_counts(norm_counts, features_genes)
        cell_label = csMAHN.pp.get_labelEncoder(reference_labels, query_labels)
        if aligned:
            n_classes = max(cell_label) + 1
        else:
            n_classes = len(np.unique(adatas[0].obs[key_class]))
        train_idx, val_idx, pred_idx = csMAHN.pp.get_idx_cross_classes(reference_labels, query_labels)
        list_idx = [train_idx, val_idx, pred_idx]
    
        def _get_nums_gene(data):
            return len(data)
    
        def _get_nums_feature_from_cell(data: Tensor):
            return data.shape[1]
    
        all_gene_nums = _get_nums_gene(nodes_genes[0]) + _get_nums_gene(nodes_genes[1])
        cell_feature_nums = _get_nums_feature_from_cell(cell_feature)
        gene_feature = torch.Tensor(csMAHN.pp.get_gene_feature(all_gene_nums, cell_feature_nums, gene_embedding))
    
        cell_gene_adj, gene_cell_adj = csMAHN.pp.get_adj_cell_gene(norm_counts[0], norm_counts[1], nodes_genes[0], nodes_genes[1])
        gene_adj = csMAHN.pp.get_adj_gene(nodes_genes, n2n,**kvarg)
        one2one_gene_nodes_net = csMAHN.pp.get_adj_gene(nodes_genes, one2one,**kvarg)
    
        inter_net = csMAHN.pp.get_pair_from_adatas(norm_counts, features_genes, N1=10, N2=10, N=5, n_jobs=32)
        acc_inter_net = (cell_label[inter_net[0]] == cell_label[inter_net[1]]).sum() / len(cell_label[inter_net[0]])
        print('Inter-net pairs ACC: {0}'.format(acc_inter_net))
        cell_adj = csMAHN.pp.get_adj_cell(scnets, inter_net) if has_mnn else get_adj_cell(scnets)
    
        # cell_adj = get_adj_cell
        
        # print graph info
        print("-------------------nodes info-------------------")
        print(f'the num of cell feats is {cell_feature_nums}')
        print(f'the num of cell nodes is {cell_feature.shape[0]}')
        print(f'the num of gene nodes is {all_gene_nums}')
        dct = dict(
            cell_feature=cell_feature,
            cell_label=cell_label,
            list_idx=list_idx,
            gene_feature=gene_feature,
    
            adj_gg=gene_adj,
            adj_cc=cell_adj,
            adj_gc=gene_cell_adj,
            adj_cg=cell_gene_adj,
        )
        # if graph_type == 'pyg':
        #     g = get_pyg(dct)
        # elif graph_type == 'dgl':
        #     g = get_dgl(dct)
        g = csMAHN.pp.get_dgl(dct)
        return g, inter_net, one2one_gene_nodes_net, cell_label, n_classes, list_idx

    csMAHN.pp.make_graph = _func
    del _func

In [4]:
df_para = pd.concat([
     pd.concat([pd.read_csv(p_cache.joinpath(
        'parameter_healthy_{}.csv'.format(i))).assign(mask=i)
        for i in 'HCL_MCA,Retina,Retina-parameters'.split(',')]),
    pd.concat([pd.read_csv(p_cache.joinpath(
        'parameter_{}.csv'.format(i)
    )).assign(mask=i) for i in 'LC,RA,Mtb,LN,COVID'.split(',')])
])

df_para['path_ref'] = df_para['path_ref'].apply(lambda x:p_cache.joinpath(x))
df_para['path_que'] = df_para['path_que'].apply(lambda x:p_cache.joinpath(x))
df_para['item'] = ut.df.apply_merge_field(df_para,'{tissue};{name_ref};{name_que}')
df_para = ut.df.reindex_with_unique_col(df_para,'item',drop=True)
ut.df.show(df_para)

,tissue,sp_ref,path_ref,name_ref,sp_simple_ref,sp_que,path_que,name_que,sp_simple_que,key_cell_type,mask
Adrenal-Gland;h_adr;m_adr,Adrenal-Gland,human,/public/workspace/licanchengup/link/res_publis...,h_adr,h,mouse,/public/workspace/licanchengup/link/res_publis...,m_adr,m,CL,HCL_MCA
Bone-Marrow;h_bon;m_bon,Bone-Marrow,human,/public/workspace/licanchengup/link/res_publis...,h_bon,h,mouse,/public/workspace/licanchengup/link/res_publis...,m_bon,m,CL,HCL_MCA


(44, 11)

In [5]:
df_para['mask'].value_counts()

mask
HCL_MCA              18
Retina-parameters     6
LN                    5
Mtb                   5
LC                    4
Retina                3
COVID                 2
RA                    1
Name: count, dtype: int64

# HCL_MCA

In [19]:
df_para_item['path_ref'].apply(lambda x:x.joinpath('matrix.mtx').exists()).all()
df_para_item['path_que'].apply(lambda x:x.joinpath('matrix.mtx').exists()).all()

True

In [21]:
item = 'HCL_MCA'
df_para_item = df_para.query("mask == '{}'".format(item))
print('\n[runstart] {}\n'.format(item))

aligned=True
is_1v1 = False
n_epochs = [100, 200, 200]
for i, row in df_para_item.iterrows():
    path_varmap = get_path_varmap(
            map_sp[row['sp_simple_ref']], map_sp[row['sp_simple_que']], model='csMAHN')
    print(path_varmap)

    run_csMAHN_info_model_gene(
        path_adata1=row['path_ref'],
        path_adata2=row['path_que'],
        key_class1=row['key_cell_type'],
        key_class2=row['key_cell_type'],
        sp1=row['sp_simple_ref'],
        sp2=row['sp_simple_que'],
        path_varmap=path_varmap,
        tissue_name=row['tissue'],
        aligned=aligned,
        resdir_tag='{}-map-{};epochs={};is_1v1={}'.format(
            row['name_ref'],
            row['name_que'],
            sum(n_epochs),
            is_1v1),
        resdir=p_res_model_gene,
        models='csMAHN,came'.split(','),
        **{'n_epochs': n_epochs,
           'is_1v1': is_1v1})

print('\n[run finish] {}\n'.format(item))


[runstart] HCL_MCA

/public/workspace/licanchengup/link/res_publish/homo/human_to_mouse.txt
[has finish]240819-0218 Adrenal-Gland_h-corss-m;h_adr-map-m_adr;epochs=500;is_1v1=False
/public/workspace/licanchengup/link/res_publish/homo/human_to_mouse.txt
[has finish]240819-0218 Bone-Marrow_h-corss-m;h_bon-map-m_bon;epochs=500;is_1v1=False
/public/workspace/licanchengup/link/res_publish/homo/human_to_mouse.txt
[start]240819-0218 Brain_h-corss-m;h_bra-map-m_bra;epochs=500;is_1v1=False
[path_varmap] /public/workspace/licanchengup/link/res_publish/homo/human_to_mouse.txt


/public/workspace/licanchengup/link/res_publish/csMAHN/utils/preprocess.py:413: FutureWarning: pandas.value_counts is deprecated and will be removed in a future version. Use pd.Series(obj).value_counts() instead.
  type_counts_list.append(pd.value_counts(adatas[i].obs[key_class]))
/public/workspace/licanchengup/link/res_publish/csMAHN/utils/preprocess.py:413: FutureWarning: pandas.value_counts is deprecated and will be removed in a future version. Use pd.Series(obj).value_counts() instead.
  type_counts_list.append(pd.value_counts(adatas[i].obs[key_class]))


----raw----
                  reference    query
CL                                  
neural cell         15645.0  26621.0
oligodendrocyte      7546.0  11170.0
macrophage           6633.0  12633.0
astrocyte            6143.0      NaN
stem cell            4596.0   7878.0
fibroblast           1382.0      NaN
granule cell          686.0      NaN
endothelial cell      386.0   2264.0
T cell                  NaN  13425.0
ciliated cell           NaN   6732.0
neutrophil              NaN   2135.0
----new----
                  reference    query
CL                                  
neural cell         15645.0  26621.0
oligodendrocyte      7546.0  11170.0
macrophage           6633.0  12633.0
stem cell            4596.0   7878.0
endothelial cell      386.0   2264.0
                  Brain_h  Brain_m
CL                                
neural cell         15645    26621
oligodendrocyte      7546    11170
macrophage           6633    12633
stem cell            4596     7878
endothelial cell      386 

/public/workspace/licanchengup/apps/miniconda3/envs/publish/lib/python3.9/site-packages/scanpy/preprocessing/_highly_variable_genes.py:216: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  disp_grouped = df.groupby('mean_bin')['dispersions']
/public/workspace/licanchengup/apps/miniconda3/envs/publish/lib/python3.9/site-packages/scanpy/preprocessing/_highly_variable_genes.py:216: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  disp_grouped = df.groupby('mean_bin')['dispersions']


the time2 of processing adatas is 17.095465183258057
Leiden results:
leiden
0     10573
1     10021
2      7048
3      5422
4      5196
5      3356
6      3153
7      2773
8      2151
9      1766
10     1616
11     1458
12     1316
13     1202
14      771
15      696
16      689
17      503
18      385
19      354
20      117
Name: count, dtype: int64
the time of leiden is 18.82611918449402


...

/public/workspace/licanchengup/link/res_publish/csMAHN/utils/preprocess.py:413: FutureWarning: pandas.value_counts is deprecated and will be removed in a future version. Use pd.Series(obj).value_counts() instead.
  type_counts_list.append(pd.value_counts(adatas[i].obs[key_class]))
/public/workspace/licanchengup/link/res_publish/csMAHN/utils/preprocess.py:413: FutureWarning: pandas.value_counts is deprecated and will be removed in a future version. Use pd.Series(obj).value_counts() instead.
  type_counts_list.append(pd.value_counts(adatas[i].obs[key_class]))


----raw----
                                                    reference    query
CL                                                                    
kidney loop of Henle ascending limb epithelial ...    25916.0      NaN
mesenchymal cell                                      19649.0  23683.0
kidney tubule cell                                    17060.0   7482.0
pancreatic ductal cell                                 4754.0   4321.0
myeloid cell                                           4431.0   2748.0
endothelial cell                                       3716.0   5147.0
epithelial cell                                         265.0    495.0
podocyte                                                  NaN   1255.0
----new----
                        reference    query
CL                                        
mesenchymal cell          19649.0  23683.0
kidney tubule cell        17060.0   7482.0
pancreatic ductal cell     4754.0   4321.0
myeloid cell               4431.0   2748.0
endothel

/public/workspace/licanchengup/apps/miniconda3/envs/publish/lib/python3.9/site-packages/scanpy/preprocessing/_highly_variable_genes.py:216: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  disp_grouped = df.groupby('mean_bin')['dispersions']
/public/workspace/licanchengup/apps/miniconda3/envs/publish/lib/python3.9/site-packages/scanpy/preprocessing/_highly_variable_genes.py:216: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  disp_grouped = df.groupby('mean_bin')['dispersions']


the time2 of processing adatas is 12.11421537399292
Leiden results:
leiden
0     9310
1     6135
2     4559
3     3999
4     3549
5     2924
6     2878
7     2473
8     2417
9     2288
10    1754
11     548
12     374
13     312
14     211
15     145
Name: count, dtype: int64
the time of leiden is 14.987849473953247


/tmp/ipykernel_227902/3021580929.py:319: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  adatas[1].obs[key_clust] = clust_lbs2
/public/workspace/licanchengup/apps/miniconda3/envs/publish/lib/python3.9/site-packages/numpy/core/fromnumeric.py:86: FutureWarning: The behavior of DataFrame.sum with axis=None is deprecated, in a future version this will reduce over both axes and return a scalar. To retain the old behavior, pass axis=0 (or do not pass axis)
  return reduction(axis=axis, out=out, **passkwargs)
/public/workspace/licanchengup/apps/miniconda3/envs/publish/lib/python3.9/site-packages/numpy/core/fromnumeric.py:86: FutureWarning: The behavior of DataFrame.sum with axis=None is deprecated, in a future version this will reduce over both axes and return a scalar. To retain the old behavior, pass axis=0 (or do not pass axis)
  return reduction(axis=axis, out=out, **passkwargs)

KeyboardInterrupt



# Retina

In [ ]:
item = 'Retina'
df_para_item = df_para.query("mask == '{}'".format(item))
print('\n[runstart] {}\n'.format(item))

aligned=True
is_1v1 = False
n_epochs = [100, 200, 200]
for is_1v1 in [False,True] :
    for i, row in df_para_item.iterrows():
        path_varmap = get_path_varmap(
                map_sp[row['sp_simple_ref']], map_sp[row['sp_simple_que']], model='csMAHN')
        print(path_varmap)
    
        run_csMAHN_info_model_gene(
            path_adata1=row['path_ref'],
            path_adata2=row['path_que'],
            key_class1=row['key_cell_type'],
            key_class2=row['key_cell_type'],
            sp1=row['sp_simple_ref'],
            sp2=row['sp_simple_que'],
            path_varmap=path_varmap,
            tissue_name=row['tissue'],
            aligned=aligned,
            resdir_tag='{}-map-{};epochs={};is_1v1={}'.format(
                row['name_ref'],
                row['name_que'],
                sum(n_epochs),
                is_1v1),
            resdir=p_res_model_gene,
            models='csMAHN,came'.split(','),
            **{'n_epochs': n_epochs,
               'is_1v1': is_1v1})

print('\n[run finish] {}\n'.format(item))

# LC

In [ ]:
item = 'LC'
df_para_item = df_para.query("tissue == '{}'".format(item))
# df_para_item = df_para.loc[['LC;LChDCs;LCmDCs'],:]
aligned=False
is_1v1 = False

print('\n[run start] {}\n'.format(item))
display(df_para_item)

n_epochs = [100, 200, 200]
for i, row in df_para_item.iterrows():
    path_varmap = get_path_varmap(
            map_sp[row['sp_simple_ref']], map_sp[row['sp_simple_que']], model='csMAHN')
    print(path_varmap)

    run_csMAHN_info_model_gene(
        path_adata1=row['path_ref'],
        path_adata2=row['path_que'],
        key_class1=row['key_cell_type'],
        key_class2=row['key_cell_type'],
        sp1=row['sp_simple_ref'],
        sp2=row['sp_simple_que'],
        path_varmap=path_varmap,
        tissue_name=row['tissue'],
        aligned=aligned,
        resdir_tag='{}-map-{};epochs={};is_1v1={}'.format(
            row['name_ref'],
            row['name_que'],
            sum(n_epochs),
            is_1v1),
        resdir=p_res_model_gene,
        models='csMAHN,came'.split(','),
        **{'n_epochs': n_epochs,
           'is_1v1': is_1v1})

print('\n[run finish] {}\n'.format(item))

# RA

In [ ]:
item = 'RA'
df_para_item = df_para.query("tissue == '{}'".format(item))
aligned=False
is_1v1 = False

print('\n[run start] {}\n'.format(item))
display(df_para_item)

n_epochs = [100, 100, 100]
for i, row in df_para_item.iterrows():
    path_varmap = get_path_varmap(
            map_sp[row['sp_simple_ref']], map_sp[row['sp_simple_que']], model='csMAHN')
    print(path_varmap)

    run_csMAHN_info_model_gene(
        path_adata1=row['path_ref'],
        path_adata2=row['path_que'],
        key_class1=row['key_cell_type'],
        key_class2=row['key_cell_type'],
        sp1=row['sp_simple_ref'],
        sp2=row['sp_simple_que'],
        path_varmap=path_varmap,
        tissue_name=row['tissue'],
        aligned=aligned,
        resdir_tag='{}-map-{};epochs={};is_1v1={}'.format(
            row['name_ref'],
            row['name_que'],
            sum(n_epochs),
            is_1v1),
        resdir=p_res_model_gene,
        models='csMAHN,came'.split(','),
        **{'n_epochs': n_epochs,
           'is_1v1': is_1v1})

print('\n[run finish] {}\n'.format(item))

# LN

In [ ]:
item = 'LN'
df_para_item = df_para.query("tissue == '{}'".format(item))
aligned=False
is_1v1 = False

print('\n[run start] {}\n'.format(item))
display(df_para_item)

n_epochs = [100, 200, 200]
for i, row in df_para_item.iterrows():
    path_varmap = get_path_varmap(
            map_sp[row['sp_simple_ref']], map_sp[row['sp_simple_que']], model='csMAHN')
    print(path_varmap)

    run_csMAHN_info_model_gene(
        path_adata1=row['path_ref'],
        path_adata2=row['path_que'],
        key_class1=row['key_cell_type'],
        key_class2=row['key_cell_type'],
        sp1=row['sp_simple_ref'],
        sp2=row['sp_simple_que'],
        path_varmap=path_varmap,
        tissue_name=row['tissue'],
        aligned=aligned,
        resdir_tag='{}-map-{};epochs={};is_1v1={}'.format(
            row['name_ref'],
            row['name_que'],
            sum(n_epochs),
            is_1v1),
        resdir=p_res_model_gene,
        models='csMAHN,came'.split(','),
        **{'n_epochs': n_epochs,
           'is_1v1': is_1v1})

print('\n[run finish] {}\n'.format(item))